In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from typing import Union, List, Callable, Optional, Dict, Any
import json
from utils import get_colors


# Sorbetto: a Python Library to Produce Classification Tiles With Different Flavors

Code in [![GitHub stars](https://img.shields.io/github/stars/pierard/sorbetto?style=social)](https://github.com/pierard/sorbetto)

## App developper

It is natural to desire to understand the theoretical bases of scores used when evaluating and ranking performances.

In this example, it is shown how the tile can be used for such objective.


For more information, please refer to the  [*reference paper*](https://arxiv.org/abs/2412.04377).

### Installing the library

If you are using google colab, it is possible to install the library by running the code below. If it is not the case, please refer to install.md to get a local installation.

In [ ]:
# only for google colab

CODE_DIR = 'sorbetto'
os.makedirs(f'./{CODE_DIR}')
!git clone git@github.com:pierard/sorbetto.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')

!bash install.sh

FileExistsError: [Errno 17] File exists: './sorbetto'

### Data

In this example, the list of entities are the different deep learning models from mmsegmentation that are obtained on cityscape data.

In [ ]:
# loading data

from sorbetto.core.entity import Entity
from sorbetto.performance.two_class_classification import TwoClassClassificationPerformance

PATH_DATA = os.path.join(os.getcwd(), 'data', 'semantic_segmentation_methods.json')

MAP_DATASETS = {
    'Cityscapes': '♠',
    'ADE20K': '♥',
    'Pascal VOC 2012': '♦',
    'COCO-Stuff 164k': '♣'
}

def read_json(filePath:str):
    """
    Function to read json 
    """
    
    with open(filePath, 'r') as f:
        data = json.load(f)
    return data


def read_data(filePath:str = PATH_DATA):
    data = read_json(filePath)

    processed_data = []

    colors = get_colors(len(data))

    # convert each instance into an Entity
    for i, elem in enumerate(data):
        performance = TwoClassClassificationPerformance(
            ptp=elem['proba_tp'],
            ptn=elem['proba_tn'],
            pfp=elem['proba_fp'],
            pfn=elem['proba_fn']
        )

        entity = Entity(
            performance= performance,
            name = f"{elem['model']} {MAP_DATASETS[elem['learning_set']]}",
            color= colors[i]
        )

        processed_data.append(entity)

    return processed_data

data = read_data()

In [ ]:
from sorbetto.analyses.analysis_app_developer import AnalysisForAppDeveloper
from sorbetto.parametrization.parametrization_default import ParameterizationDefault

min_a = ...
max_a = ...
min_b = ...
max_b = ...
performances = ...
parametrization = ParameterizationDefault()
resolution = 1001

analysis_dev = AnalysisForAppDeveloper(
    min_a=min_a,
    max_a=max_a,
    min_b=min_b,
    max_b=max_b,
    performances=performances,
    parametrization=parametrization,
    resolution=resolution
)

### Entity Tile

In [ ]:
rank = ...
entity_tile = analysis_dev.getEntityTile()

### Value Tile

In [ ]:
rank = ...
entity_tile = analysis_dev.getEntityTile()

### Advice

In [ ]:
rank = ...
entity_tile = analysis_dev.getEntityTile()